In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json as js
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec



In [14]:
#load pickle file
df = pd.read_pickle('data/cleaned_elden_ring_data.pkl')


In [15]:
df.tail(20)


,name,text,type
8321,End of Game Dialog6,The fallen leaves tell a story.,dialog
8322,End of Game Dialog7,"Of how a Tarnished, of no renown, became Elden...",dialog
8323,End of Game Dialog8,The fallen leaves tell a story. Hrghraah! The ...,dialog
8324,End of Game Dialog9,Of how a Tarnished became Elden Lord. Have it ...,dialog
8325,End of Game Dialog10,"In our home, across the fog, the Lands Between...",dialog
8326,End of Game Dialog11,...,dialog
8327,End of Game Dialog12,"Tarnished, thou'rt but a fool. The Erdtree war...",dialog
8328,End of Game Dialog13,,dialog
8329,End of Game Dialog14,None may claim the title of Elden Lord. Thy de...,dialog
8330,End of Game Dialog15,,dialog


In [65]:
with open('/Users/divygobiraj/Desktop/projects/ER_Lore_Bot/ER-lore-bot/data_processing/data/cleaned_elden_ring_data.json') as fp:
    data = js.load(fp)

In [59]:
data

[{'name_en': 'Petition for Help',
  'text': 'Online multiplayer item. Receipt of a plea for help to the maidens of the Finger Reader.  Summons a Broken Finger Stalker from another world to face an invading Broken Finger.  Maidens of the Finger Reader speak in hushed tones about the loathsome, traitorous Broken Fingers and the dangers of their base invasions. Summons Stalker to face invading Broken Finger ',
  'type': 'accessories'},
 {'name_en': 'Broken Finger Stalker Contract',
  'text': 'Online multiplayer item. Record of contract with the maidens of the Finger Reader.  Be summoned to the world of a player who has been invaded by a Broken Finger.  The Broken Fingers have betrayed the Finger Reader. They are a cancer that afflicts the ties that bind the Tarnished to one another. They must be excised, and eradicated. Be summoned to worlds invaded by Broken Fingers ',
  'type': 'accessories'},
 {'name_en': 'Crimson Amber Medallion',
  'text': "A medallion with crimson amber inlaid. Boos

In [22]:


load_dotenv(dotenv_path="/Users/divygobiraj/Desktop/projects/ER_Lore_Bot/ER-lore-bot/.env")
# print(os.getenv("PINECONE_KEY"))

pc = Pinecone(api_key=os.getenv("PINECONE_KEY"))
index = pc.Index("er-text")


In [36]:
def embed_text(text):
    return pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[text],
        parameters={"input_type": "passage", "truncate": "END"}
    )


chunked_vectors = []
batch_size = 200
for i in range(0, len(df), 200):
    vectors = []
    keepgo = 1
    while i % 200 > 0 or keepgo == 1:
        row = df.iloc[i]
        print(i)
        keepgo = 0
        vectors.append(
            {
                "id": row['name'],
                "values": (embed_text(row['name'] + ": " +  row['text'])).data[0].values, 
                "metadata": {"type": row['type']}
            }
        )
        i+=1
    chunked_vectors.append(vectors)
    vectors = []
    keepgo = 1





    




0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

IndexError: single positional indexer is out-of-bounds

In [31]:
row = df.iloc[0]
print(embed_text(row['name'] + ": " +  row['text']).data[0].values)

[0.042144775390625, -0.005229949951171875, -0.03900146484375, -0.035125732421875, 0.034423828125, -0.042877197265625, -0.003597259521484375, 0.0550537109375, 0.056854248046875, -0.0113677978515625, 0.0341796875, 0.04730224609375, -0.03900146484375, -0.035614013671875, -0.0121307373046875, 0.01081085205078125, -0.007701873779296875, 0.01190948486328125, 0.0103759765625, -0.0221710205078125, 0.031402587890625, 0.004673004150390625, -0.041229248046875, -0.021820068359375, -0.02197265625, -0.013031005859375, -0.031646728515625, -0.03741455078125, -0.0007567405700683594, -0.0300445556640625, -0.0136566162109375, -0.0146636962890625, 0.005321502685546875, -0.01153564453125, -0.0130767822265625, 0.01059722900390625, 0.052154541015625, 0.034820556640625, -0.047943115234375, 0.015228271484375, -0.045379638671875, 0.042877197265625, -0.03704833984375, -0.038330078125, 0.006130218505859375, 0.005153656005859375, 0.0276336669921875, -0.0200042724609375, 0.021026611328125, 0.02581787109375, 0.01535

In [40]:
for vec in chunked_vectors:
    index.upsert(vectors=vec)


200